<a href="https://colab.research.google.com/github/engineer-nicolas/cs50sql/blob/master/lecture_3_Writing/lecture_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 3 - Writing - CS50 SQL harvard

https://cs50.harvard.edu/sql/notes/3/


In this notebook, it will be explored how to add, update, and delete data in databases.

The Boston MFA (Museum of Fine Arts) is a century-old museum in Boston. The MFA manages a vast collection of historical and contemporary artifacts and artwork. We will focus now on the creation (or insertion) of data in a Boston MFA database.

In [1]:
# Let's import the required libraries
import sqlite3
import pandas as pd
import os

In [11]:
#Delete only if it already exists
DB_NAME = "mfa.db"

# Safety check: delete if the filename is exactly what we expect
if os.path.exists(DB_NAME) and DB_NAME.endswith(".db"):
    os.remove(DB_NAME)

Let's create a database called mfa.db.

In [12]:
# Connect to the database file
# it will create the file if it doesn't exist
conn = sqlite3.connect('mfa.db')
# MFA stands for the Boston Museum of Fine Arts (MFA)


In [13]:
conn.execute("""
CREATE TABLE "collections" (
    "id" INTEGER,
    "title" TEXT NOT NULL,
    "accession_number" TEXT NOT NULL UNIQUE,
    "acquired" NUMERIC,
    PRIMARY KEY("id")
);
""")

# Commit changes to save the table in the database
conn.commit()

# Let's inspect the schema of the collections table
pd.read_sql_query(
    "PRAGMA table_info(collections);",
    conn
)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,title,TEXT,1,None,0
2,2,accession_number,TEXT,1,None,0
3,3,acquired,NUMERIC,0,None,0


## INSERT INTO ___ VALUES ___


This SQL statement is used to insert a row of data into a given table.



In [14]:
# Execute the INSERT statement
conn.execute('''
INSERT INTO "collections" ("id", "title", "accession_number", "acquired")
VALUES (9, 'Profusion of flowers', '56.257', '1956-04-12');
''')

# Save changes to the database
conn.commit()

pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,9,Profusion of flowers,56.257,1956-04-12


This command requires the list of columns in the table that will receive new data and the values to be added to each column, in the *same order*.

IMPORTANT: SQLite can fill out the primary key values automatically. To make use of this functionality, we omit the ID column altogether while inserting a row.

In [15]:
# Execute the INSERT statement
conn.execute('''
INSERT INTO "collections" ("title", "accession_number", "acquired")
VALUES ('Farmers working at dawn', '11.6152', '1911-08-03');
''')

# Save changes to the database
conn.commit()

pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,9,Profusion of flowers,56.257,1956-04-12
1,10,Farmers working at dawn,11.6152,1911-08-03


SQLite selects the highest primary key value in the table and increments it to generate the next primary key value — in this case, 10.

### Inserting Multiple Rows

Let us now insert two new paintings into the collections table.

In [16]:
# Execute the INSERT statement
conn.execute('''
INSERT INTO "collections" ("title", "accession_number", "acquired")
VALUES
('Imaginative landscape', '56.496', NULL),
('Peonies and butterfly', '06.1899', '1906-01-01');
''')

# Save changes to the database
conn.commit()

pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,9,Profusion of flowers,56.257,1956-04-12
1,10,Farmers working at dawn,11.6152,1911-08-03
2,11,Imaginative landscape,56.496,None
3,12,Peonies and butterfly,06.1899,1906-01-01


## IMPORT CSV FILE

SQLite makes it possible to import a CSV file directly into our database. We can import the CSV by running a SQLite command:

`.import --csv --skip 1 mfa.csv collections`

The first argument, --csv indicates to SQLite that we are importing a CSV file.

The second argument indicates that the first row of the CSV file (the header row) needs to be skipped, or not inserted into the table.




## INSERT INTO ___ + SELECT ___ FROM - Inserting from another table

In [31]:
csv_file = """title,accession_number,acquired
Profusion of flowers,56.257,1956-04-12
Farmers working at dawn,11.6152,1911-08-03
Spring outing,14.769,1914-01-08
Red landscape,56.476,
Peonies and butterfly,06.1899,1906-01-01
"""

with open("temp.csv", "w", encoding="utf-8") as f:
    f.write(csv_file)

# Read the CSV file into a DataFrame
df = pd.read_csv("temp.csv")
print(df)

                     title  accession_number    acquired
0     Profusion of flowers           56.2570  1956-04-12
1  Farmers working at dawn           11.6152  1911-08-03
2            Spring outing           14.7690  1914-01-08
3            Red landscape           56.4760         NaN
4    Peonies and butterfly            6.1899  1906-01-01


In [32]:
conn.execute('''
CREATE TABLE IF NOT EXISTS temp (
    title TEXT,
    accession_number TEXT,
    acquired NUMERIC
);
''')
conn.commit()

df.to_sql(
    'temp',
    conn,
    if_exists='replace',
    index=False
)

5

In [22]:
pd.read_sql_query('SELECT * FROM temp;', conn)


,title,accession_number,acquired
0,Profusion of flowers,56.2570,1956-04-12
1,Farmers working at dawn,11.6152,1911-08-03
2,Spring,14.7690,1914-01-08
3,Red landscape,56.4760,None
4,Peonies and butterfly,6.1899,1906-01-01


In [23]:
conn.execute("""
INSERT OR IGNORE INTO "collections" ("title", "accession_number", "acquired")
SELECT "title", "accession_number", "acquired"
FROM "temp";
""")
conn.commit()
pd.read_sql_query(
    'SELECT * FROM collections;',
    conn
)


,id,title,accession_number,acquired
0,9,Profusion of flowers,56.257,1956-04-12
1,10,Farmers working at dawn,11.6152,1911-08-03
2,11,Imaginative landscape,56.496,None
3,12,Peonies and butterfly,06.1899,1906-01-01
4,13,Spring,14.769,1914-01-08
5,14,Red landscape,56.476,None
6,15,Peonies and butterfly,6.1899,1906-01-01


## DELETE

## DELETE FROM - Deleting all rows

Running the following command would delete all rows from the table `collections`

In [38]:
conn.execute('''
DELETE FROM "collections";
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired


In [39]:
conn.execute("""
INSERT OR IGNORE INTO "collections" ("title", "accession_number", "acquired")
SELECT "title", "accession_number", "acquired"
FROM "temp";
""")
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,1,Profusion of flowers,56.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03
2,3,Spring outing,14.769,1914-01-08
3,4,Red landscape,56.476,None
4,5,Peonies and butterfly,6.1899,1906-01-01


## DELETE FROM ___ + WHERE ___ - Deleting selected rows

We can also delete rows that match specific conditions.

For example, to delete the painting “Spring outing” from our table collections we can run:

In [40]:
conn.execute('''
DELETE FROM "collections"
WHERE "title" = 'Spring outing';
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,1,Profusion of flowers,56.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03
2,4,Red landscape,56.476,None
3,5,Peonies and butterfly,6.1899,1906-01-01


To delete paintings older than 1909, we can run

In [41]:
conn.execute('''
DELETE FROM "collections"
WHERE "acquired" < '1909-01-01';
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,1,Profusion of flowers,56.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03
2,4,Red landscape,56.476,None


## DELETE FROM ___ + WHERE ___ IS NULL - Deleting rows with NULL values

In [42]:
conn.execute('''
DELETE FROM "collections"
WHERE "acquired" IS NULL;
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,1,Profusion of flowers,56.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03


## UPDATE ___ SET ___ = ___ WHERE ___

Here is the syntax of the update command:

> UPDATE table
> SET column0 = value0, column1 = value1, ...
> WHERE condition;

Let's change the title of 'Profusion of flowers' painting to 'Flowers':

In [43]:
conn.execute('''
UPDATE "collections" 
SET "title"='Flowers',"accession_number"='5.257'
WHERE "title"='Profusion of flowers';
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,1,Flowers,5.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03


## Triggers

A trigger is a SQL statement that runs automatically in response to another SQL statement, such as an INSERT, UPDATE, or DELETE.

Triggers are useful for maintaining data consistency and automating tasks across related tables.

Let's create a new `transactions` table.

In [ ]:
conn.execute("""
CREATE TABLE "transactions" (
    "id" INTEGER,
    "title" TEXT,
    "action" TEXT,
    PRIMARY KEY("id")
);
""")

# Commit changes to save the table in the database
conn.commit()
# Let's inspect the schema of the transactions table
pd.read_sql_query(
    "PRAGMA table_info(transactions);",
    conn
)

## CREATE TRIGGER ___ BEFORE ___ ON___ BEGIN ___ END; + DELETE/UPDATE + OLD

IMPORTANT: `OLD` is a special keyword that refers to the row being deleted or updated. If this row had a title column, `OLD."title"` would access that column’s value.

When artwork is sold and deleted from `collections`, we want it automatically logged in `transactions` with an action of “sold”.

In [ ]:
# Create the trigger
conn.execute('''
CREATE TRIGGER "sell"
BEFORE DELETE ON "collections"
BEGIN
    INSERT INTO "transactions" ("title", "action")
    VALUES (OLD."title", 'sold');
END;
''')

# Save changes
conn.commit()
pd.read_sql_query("""
SELECT *
FROM sqlite_master
WHERE type = 'trigger';
""", conn)

Let’s test the trigger:

In [ ]:
# Execute the INSERT statement
conn.execute("""
INSERT INTO "collections" ("title", "accession_number", "acquired")
VALUES
('Flowers', '56.493', NULL);
""")

# Execute the DELETE statement
conn.execute("""
DELETE FROM "collections"
WHERE "title" = 'Flowers';
""")
# Save changes to the database
conn.commit()

In [ ]:
pd.read_sql_query("SELECT * FROM transactions;", conn)

## CREATE TRIGGER ___ AFTER ___ ON ___ + INSERT/UPDATE + NEW

IMPORTANT: `NEW` is a special keyword that refers to the row being inserted or updated. If this row has a title column, `NEW."title"` would accesses that column’s value.


When artwork is bought (inserted into collections), we want it logged in transactions with an action of “bought”.

In [ ]:
# Create the trigger
conn.execute("""
CREATE TRIGGER "buy"
AFTER INSERT ON "collections"
BEGIN
    INSERT INTO "transactions" ("title", "action")
    VALUES (NEW."title", 'bought');
END;
""")

# Save changes
conn.commit()
pd.read_sql_query("""
SELECT *
FROM sqlite_master
WHERE type = 'trigger';
""", conn)

Let’s test the trigger:

In [ ]:
# Execute the INSERT statement
conn.execute("""
INSERT INTO "collections" ("title", "accession_number", "acquired")
VALUES
('Flowers', '56.493', NULL);
""")

# Execute the DELETE statement
conn.execute("""
DELETE FROM "collections"
WHERE "title" = 'Flowers';
""")
# Save changes to the database
conn.commit()

In [ ]:
pd.read_sql_query("SELECT * FROM transactions;", conn)

## Soft Deletions

Soft deletion (or a soft delete) means marking data as deleted rather than actually removing it from the database.


For example, we could add a `deleted` column to the `collections` table with a default value of 0. To “delete” a row, we would update the deleted column to 1. This way, data can be recovered if needed and maintains a complete historical record.

In [ ]:
# Add "deleted" column
conn.execute('''
ALTER TABLE "collections"
ADD COLUMN "deleted" INTEGER DEFAULT 0;
''')

# Save changes to the database
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

To “delete” a row, we would update the deleted column to 1:

In [ ]:
# Update "deleted" column without deleting row
conn.execute('''
UPDATE "collections"
SET "deleted"=1
WHERE "title"='Farmers working at dawn';
''')

# Save changes to the database
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

Then, to query only non-deleted rows:

In [ ]:
pd.read_sql_query("""
SELECT * FROM "collections"
WHERE "deleted" != 1;
""", conn)

This way, data can be recovered if needed and maintains a complete historical record.